In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from PIL import Image
import os
from pathlib import Path
from utils import *
from allResNets import *
from engine import *
from data_transforms import create_transform
from dataloaders import *

change_to_disk()
data_dir = Path("clean_split_1k/")
models_path = Path('Quality/Models_test/')
check_dir(models_path)

In [2]:
image_shape = (1080, 1920)
resize_factor = 0.1
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]
BATCH_SIZE = 16

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders_multilabel(data_dir, transform = data_transform, batch_size=BATCH_SIZE, num_workers= NUM_WORKERS)

In [3]:
NUM_EPOCHS = 2
lr = 0.001
device = get_device()

resnet18 = ResNet18()

#Changed in order to fit the multilabel problem
loss_fn = nn.BCEWithLogitsLoss()

In [4]:
model_name = "ResNet18"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet18
resnet18.to(device)
nadam_optim = torch.optim.NAdam(params=resnet18.parameters(), lr=lr)
train_resnet18_results, train_time_resnet18 = train(resnet18, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.3159 | train_acc: 0.4170 | validation_loss: 0.3874 | validation_acc: 0.4180


In [ ]:
# Get a single image batch
image_batch, label_batch = next(iter(train_dl))
image_batch.shape, label_batch.shape

In [ ]:
image_batch[0], label_batch[0]

In [ ]:
resnet18.to(device)
y = resnet18(image_batch.to(device))

In [ ]:
print(y)

In [ ]:
label_batch

In [ ]:
y_pred_class = torch.round(torch.sigmoid(y))
y_pred_class